In [1]:
# you can skip it, I just like interactive charts
%matplotlib Qt

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.wcs import WCS
import pandas as pd
# nice progressbar
from tqdm import tqdm

In [11]:
data = pd.read_csv("gaia.csv")
data = data.dropna(subset=['Gmag'])
mask = (data['Gmag'] > 13) & (data['Gmag'] < 20)
data = data[mask]
coords = SkyCoord(ra=data["RA_ICRS"], dec=data["DE_ICRS"], frame='icrs', unit=u.degree)

#### Grid

I create a rectangular grid. The boundaries are minimal and maximal RA and DEC from the given catalogue.

In [12]:
grid_step = 1 # in arcmin
ra_grid = np.arange(coords.ra.min().to(u.arcmin).value, coords.ra.max().to(u.arcmin).value, grid_step)
dec_grid = np.arange(coords.dec.min().to(u.arcmin).value, coords.dec.max().to(u.arcmin).value, grid_step)
ra_v, dec_v = np.meshgrid(ra_grid, dec_grid)
# now grid is a 2-d array, each element is SkyCoord (ra,dec)
grid = SkyCoord(ra_v, dec_v, unit=u.arcmin)

In [13]:
def create_wcs(center: SkyCoord, cdelt=1*u.arcmin, size=1*u.deg):
    pixsize = int((size.to(cdelt.unit)/cdelt).value)
    
    wcs = WCS(naxis=2)
    wcs.wcs.cdelt = [cdelt.value, cdelt.value]
    wcs.wcs.cunit = [cdelt.unit, cdelt.unit]
    wcs.wcs.ctype = ["RA", "DEC"]
    wcs.wcs.crpix = [pixsize/2., pixsize/2.]
    wcs.wcs.crval = [center.ra.to(cdelt.unit).value, center.dec.to(cdelt.unit).value]
    wcs.pixel_shape = [pixsize, pixsize]
    return(wcs)
# self.slit_wcs.wcs.latpole = 0.
# self.slit_wcs.wcs.lonpole = 180.
# self.slit_wcs.wcs.equinox = 2000.0

# w_header = self.slit_wcs.to_header()
# w_header['NAXIS'] = 2
# w_header['NAXIS1'], w_header['NAXIS2'] = self.slit_wcs.pixel_shape

In [26]:
ra_grid.mean()

np.float64(3140.2595965066)

In [27]:
center = SkyCoord('03:29:11.28', '+31:18:36.00', unit=(u.hourangle, u.deg))
center = SkyCoord(ra_grid.mean(), dec_grid.mean(), unit=(u.arcsec, u.arcsec))
cdelt = 1 * u.arcmin
wcs = create_wcs(center, cdelt)

#### Sourse density map

In [15]:
density = np.zeros_like(grid, dtype=float)

imax, jmax = grid.shape

# it takes about 40s on my computer with density_window=5
density_window = 10*u.arcmin # diameter of region in which we calculate density
for i in tqdm(range(imax)):
    for j in range(jmax):
        mask = grid[i,j].separation(coords) < density_window
        density[i,j] = len(mask[mask])

100%|█████████████████████████████████████████| 120/120 [00:27<00:00,  4.41it/s]


In [28]:
# need to inverse array so that it cossesponds to the aladin axis orientation
fig, ax = plt.subplots(subplot_kw=dict(projection=wcs))
# ax.imshow(density[:,::-1], origin='lower')
ax.imshow(density, origin='lower')

In [17]:
density_dis = density.flatten()
density_dis = density_dis[density_dis !=0]
n, num = np.histogram(density_dis, bins=10)
num = (num[:-1] + num[1:])/2
plt.figure()
plt.plot(num, n)

### Select region

In [23]:
def stars_from_region(data, coords, center, size):
    mask = center.separation(coords) < size
    return data[mask], coords[mask]

## Source histogram

In [22]:
N, m = np.histogram(data['Gmag'], bins=50)
# N = np.cumsum(N)
m = (m[:-1] + m[1:])/2
# p = np.polynomial.Polynomial.fit(m, np.log(N), 2)
# # err = np.sqrt(np.diag(cov))
# p(m)
print(N)

[ 19  13  18  23  22  24  28  35  38  37  49  56  47  65  62  77  85  86
  87 106 112 122 157 172 144 185 174 200 234 244 229 278 285 309 337 364
 374 435 423 446 471 489 546 538 619 595 668 663 692 727]


In [21]:
plt.plot(m, np.log(N))
# plt.plot(m, p(m))

,_RAJ2000,_DEJ2000,_V,RA_ICRS,DE_ICRS,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,...,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,2MASS,RAVE6,RAJ2000,DEJ2000
1,52.431856,30.288244,VizieR,52.431868,30.288222,120716446766073088,0.5059,0.3604,0.7471,0.5421,...,,,J032943.63+301717.4,NaN,NCHD013028,,03294362+3017174,,52.431856,30.288244
2,52.356935,30.273780,VizieR,52.356951,30.273756,120716725939608960,0.0913,0.0721,0.5651,0.1016,...,,URAT1-602031070,J032925.66+301625.5,NaN,NCHD005917,,03292565+3016257,,52.356935,30.273780
3,52.325371,30.269596,VizieR,52.325350,30.269598,120716859082650240,0.0724,0.0556,1.6789,0.0800,...,,URAT1-602031056,J032918.08+301610.5,NaN,NCHD005888,,03291808+3016107,,52.325371,30.269596
4,52.311259,30.270132,VizieR,52.311272,30.270091,120716859082650368,0.2769,0.2090,0.8580,0.3174,...,,URAT1-602031050,J032914.70+301612.3,NaN,NCHD005890,,03291469+3016127,,52.311259,30.270132
5,52.344732,30.275825,VizieR,52.344734,30.275814,120716927802129536,0.1994,0.1512,0.5243,0.2293,...,,URAT1-602031065,J032922.74+301632.9,NaN,NCHD005930,,03292272+3016331,,52.344732,30.275825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19421,52.677368,32.217265,VizieR,52.677363,32.217229,124557247040836224,0.3284,0.2296,-0.7043,0.3862,...,,URAT1-612035343,J033042.56+321302.0,NaN,NCHF000768,,03304257+3213021,,52.677368,32.217265
19422,52.656175,32.216136,VizieR,52.656186,32.216130,124557285695142272,0.2580,0.1719,0.3415,0.2901,...,,URAT1-612035334,,NaN,NCHF000766,,03303752+3212581,,52.656175,32.216136
19423,52.663613,32.223805,VizieR,52.663628,32.223808,124557285695955584,0.0731,0.0500,0.6868,0.0804,...,,URAT1-612035339,J033039.26+321325.7,NaN,NCHF000781,,03303927+3213256,,52.663613,32.223805
19426,52.624198,32.227103,VizieR,52.624197,32.227080,124557453199268224,0.5107,0.3575,0.5840,0.5817,...,,,J033029.80+321337.5,NaN,NCHF000787,,,,52.624198,32.227103
